In [3]:
from datetime import timedelta
import cv2
import numpy as np
import os
import sys

In [25]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps = 1):
    """Функция, которая возвращает список длительностей, в которые следует сохранять кадры."""
    s = []
    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # используйте np.arange () для выполнения шагов с плавающей запятой
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [29]:
def frame_cut(video_file, path):
    
    if not os.path.isdir(path + video_file[:-4]):
        print('check')
        os.mkdir(path + video_file[:-4])
    else:
        return 'stop'
    
    cap = cv2.VideoCapture(video_file)

    fps = cap.get(cv2.CAP_PROP_FPS)

    print('FPS = ', round(fps, 5))
    
    saving_frames_durations = get_saving_frames_durations(cap)
    
    count = 0
    while True:
        is_read, frame = cap.read()
        print(is_read, count)
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(path + video_file + '\\' + f"frame{frame_duration_formatted}.jpg", frame)
            print('success')
            
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1


In [1]:
video_file = 'Camera 3_20220526_003249.mp4'
path = 'C:\\Users\\Reginald Miller\\Data_Science\\Tasks\\table_check\\Frames_'

###frame_cut(video_file, path)

In [27]:
from PIL import Image

In [46]:
path1 = "Frames_Camera 3_20220526_003249"
frames_f = os.listdir(path1)

# Вывести все файлы и папки
for file in frames_f:
    img = Image.open('Frames_Camera 3_20220526_003249/' + file)
    im_crop = img.crop((115, 210, 350, 445))
    im_crop = im_crop.resize((116, 116), Image.ANTIALIAS)
    im_crop.save('Base/cut116_' + file, quality=95)

<ipython-input-46-ac4bf8314357>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im_crop = im_crop.resize((116, 116), Image.ANTIALIAS)
